In [ ]:
!pip install datasets
!pip install torch
!pip install evaluate
!pip install stop_words
!pip install datasets imbalanced-learn


In [ ]:
!python -m spacy download ru_core_news_sm
!python -m spacy download fi_core_news_sm
!python -m spacy download ja_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 39.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fi_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 97.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load(

In [ ]:
#Dont know why this is coming up for me
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
!pip uninstall wandb

Found existing installation: wandb 0.18.5
Uninstalling wandb-0.18.5:
  Would remove:
    /usr/local/bin/wandb
    /usr/local/bin/wb
    /usr/local/lib/python3.10/dist-packages/package_readme.md
    /usr/local/lib/python3.10/dist-packages/wandb-0.18.5.dist-info/*
    /usr/local/lib/python3.10/dist-packages/wandb/*
Proceed (Y/n)? Y
y
  Successfully uninstalled wandb-0.18.5


In [ ]:
#imports
import pandas as pd
import datasets
from datasets import Dataset
from datasets import load_dataset
import evaluate
import numpy as np
import nltk
from tqdm.notebook import tqdm
import spacy
from transformers import BertTokenizer
from transformers import pipeline, Trainer, TrainingArguments
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import mean_squared_error
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import stop_words
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Placing the model on the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
#Loading dataset
ds = load_dataset("coastalcph/tydi_xor_rc")
train = ds['train']
validation = ds['validation']

In [ ]:
# Seperating by languages

# Training: Seperating by language
train_ru_df = train.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'ru')
train_ja_df = train.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'ja')
train_fi_df = train.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'fi')
train_en_df = train.select_columns([ 'context','lang', 'answerable'])

# Testing
val_ru_df = validation.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'ru')
val_ja_df = validation.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'ja')
val_fi_df = validation.select_columns(['question', 'context','lang', 'answerable']).filter(lambda x: x['lang'] == 'fi')
val_en_df = validation.select_columns([ 'context','lang', 'answerable'])

In [ ]:
# Creating a function to change boolean label into 0 and 1

def prediction_change(dataset, column_name):
  dataset['labels'] = dataset[column_name] * 1
  return dataset


In [ ]:
# Applying prediction_change method

# Training
train_ru_df = train_ru_df.map(lambda x: prediction_change(x, 'answerable'))
train_ja_df = train_ja_df.map(lambda x: prediction_change(x, 'answerable'))
train_fi_df = train_fi_df.map(lambda x: prediction_change(x, 'answerable'))
train_en_df = train_en_df.map(lambda x: prediction_change(x, 'answerable'))

#Validation
val_ru_df = val_ru_df.map(lambda x: prediction_change(x, 'answerable'))
val_ja_df = val_ja_df.map(lambda x: prediction_change(x, 'answerable'))
val_fi_df = val_fi_df.map(lambda x: prediction_change(x, 'answerable'))
val_en_df = val_en_df.map(lambda x: prediction_change(x, 'answerable'))

In [ ]:
# Dropping unnecissary columns

# Train
train_ru_df = train_ru_df.remove_columns(['lang','answerable' ])
train_ja_df = train_ja_df.remove_columns(['lang','answerable'])
train_fi_df = train_fi_df.remove_columns(['lang','answerable'])
train_en_df = train_en_df.remove_columns(['lang','answerable'])

# Validation
val_ru_df = val_ru_df.remove_columns(['lang','answerable'])
val_ja_df = val_ja_df.remove_columns(['lang','answerable'])
val_fi_df = val_fi_df.remove_columns(['lang','answerable'])
val_en_df = val_en_df.remove_columns(['lang','answerable'])

# Roberta Model

In [ ]:
#Loading ROBERTA
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# Loading pre-trained XLM-R
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2) #Two labels for classification
model.gradient_checkpointing_enable()
#putting model to device for storage reasons
model.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

## Tokenizer Functions

In [ ]:
# Creating a tokenizer funtion

def tokenize(dataset):
    tokenized_dataset = tokenizer(dataset['context'],dataset['question'], padding=True, truncation=True, max_length=512)
    return tokenized_dataset

In [ ]:
# Creating a tokenizer English funtion

def tokenize_en(dataset):
    tokenized_dataset = tokenizer(dataset['context'], padding=True, truncation=True, max_length=512)
    return tokenized_dataset

In [ ]:
# Tokenizing context and questions

#Training
tokenized_train_ru = train_ru_df.map(tokenize, batched=True)
tokenized_train_ja = train_ja_df.map(tokenize, batched=True)
tokenized_train_fi  = train_fi_df.map(tokenize, batched=True)
tokenized_train_en = train_en_df.map(tokenize_en, batched=True)

#Validation
tokenized_eval_ru = val_ru_df.map(tokenize, batched=True)
tokenized_eval_ja = val_ja_df.map(tokenize, batched=True)
tokenized_eval_fi  = val_fi_df.map(tokenize, batched=True)
tokenized_eval_en = val_en_df.map(tokenize_en, batched=True)


## Metrics and Training Argumenets

In [ ]:
# Training arguments

train_args = TrainingArguments(output_dir="my_trainer",
                                  eval_strategy="steps",
                                  num_train_epochs=8.0,
                                  per_device_train_batch_size=16,
                                  learning_rate=2e-5,
                                  eval_steps=500)

In [ ]:
# Loading metrics
accuracy_metric = evaluate.load('accuracy')
precision_metric = evaluate.load('precision')
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load('f1')

# Function to compute the metrics
def compute_metrics(eval_pred):
    outputs, labels = eval_pred
    predictions = np.argmax(outputs, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'f1': f1['f1']
    }

## Russian language


In [ ]:
# Checking the balance

le = LabelEncoder()
labels_encoded = le.fit_transform(tokenized_train_ru['labels'])
unique, counts = np.unique(labels_encoded, return_counts=True)
class_distribution = dict(zip(le.inverse_transform(unique), counts))
print(class_distribution)

{0: 233, 1: 1750}


In [ ]:
#Oversampling minory class

df = pd.DataFrame(tokenized_train_ru)

# Counting the occurrences of each label
class_counts = df['labels'].value_counts()

majority_label = class_counts.idxmax()
minority_label = class_counts.idxmin()

n_minority_samples = class_counts[majority_label] - class_counts[minority_label]

# Oversampling the minority class
minority_samples = df[df['labels'] == minority_label]
oversampled_minority_samples = minority_samples.sample(n=n_minority_samples, replace=True, random_state=42)

# Combining datasets
balanced_df = pd.concat([df, oversampled_minority_samples])

balanced_dataset = Dataset.from_pandas(balanced_df)

# Checking balance
new_label_counts = balanced_df['labels'].value_counts()
print(new_label_counts)

labels
1    1750
0    1750
Name: count, dtype: int64


In [ ]:
# Initializing the trainer
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset= balanced_dataset,
    eval_dataset= tokenized_eval_ru,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)



In [ ]:
trainer.train()


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,No log,0.416658,0.833333,0.827851,0.833333,0.827998
400,No log,0.505685,0.893939,0.893051,0.893939,0.890544
600,0.327800,0.480305,0.909091,0.910035,0.909091,0.905835
800,0.327800,0.757112,0.886364,0.886258,0.886364,0.881621
1000,0.072100,0.598759,0.926768,0.927337,0.926768,0.924825
1200,0.072100,0.721129,0.916667,0.915893,0.916667,0.915032


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
200,No log,0.416658,0.833333,0.827851,0.833333,0.827998
400,No log,0.505685,0.893939,0.893051,0.893939,0.890544
600,0.327800,0.480305,0.909091,0.910035,0.909091,0.905835
800,0.327800,0.757112,0.886364,0.886258,0.886364,0.881621
1000,0.072100,0.598759,0.926768,0.927337,0.926768,0.924825
1200,0.072100,0.721129,0.916667,0.915893,0.916667,0.915032
1400,0.072100,0.851418,0.898990,0.900635,0.898990,0.894569
1600,0.007900,0.845522,0.901515,0.902973,0.901515,0.897405


TrainOutput(global_step=1752, training_loss=0.11682092639953578, metrics={'train_runtime': 2940.3069, 'train_samples_per_second': 9.523, 'train_steps_per_second': 0.596, 'total_flos': 7367109550080000.0, 'train_loss': 0.11682092639953578, 'epoch': 8.0})

In [ ]:
# Print results
eval_results = trainer.evaluate()
print(f'Langugae ru: {eval_results}')

Langugae ru: {'eval_loss': 0.8289397358894348, 'eval_accuracy': 0.898989898989899, 'eval_precision': 0.8997769906860815, 'eval_recall': 0.898989898989899, 'eval_f1': 0.894976794976795, 'eval_runtime': 12.0966, 'eval_samples_per_second': 32.736, 'eval_steps_per_second': 4.133, 'epoch': 8.0}


In [ ]:
# Predictions
outputs = trainer.predict(tokenized_eval_ru)
logits = outputs.predictions
labels = outputs.label_ids
predictions = np.argmax(logits, axis=-1)

print("Predictions:", predictions)
print("True Labels:", labels)

# Confusion matrix
conf_matrix = confusion_matrix(labels, predictions)

#print("Confusion Matrix:")
#print(conf_matrix)

Predictions: [1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 0 0 1 1 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0
 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1
 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0]
True Labels: [1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1

## Finnish Language


In [ ]:
#Checking the balance

le = LabelEncoder()
labels_encoded = le.fit_transform(tokenized_train_fi['labels'])
unique, counts = np.unique(labels_encoded, return_counts=True)
class_distribution = dict(zip(le.inverse_transform(unique), counts))
print(class_distribution)

{0: 263, 1: 1863}


In [ ]:
#Oversampling minory class

df = pd.DataFrame(tokenized_train_fi)

# Counting the occurrences of each label
class_counts = df['labels'].value_counts()

majority_label = class_counts.idxmax()
minority_label = class_counts.idxmin()

n_minority_samples = class_counts[majority_label] - class_counts[minority_label]

# Oversampling the minority class
minority_samples = df[df['labels'] == minority_label]
oversampled_minority_samples = minority_samples.sample(n=n_minority_samples, replace=True, random_state=42)

# Combining datasets
balanced_df = pd.concat([df, oversampled_minority_samples])

balanced_dataset = Dataset.from_pandas(balanced_df)

# Checking balance
new_label_counts = balanced_df['labels'].value_counts()
print(new_label_counts)

labels
1    1863
0    1863
Name: count, dtype: int64


In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset= balanced_dataset,
    eval_dataset= tokenized_eval_fi,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.690300,0.758668,0.280303,0.078570,0.280303,0.122736
1000,0.466200,0.588965,0.816288,0.819136,0.816288,0.817556


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.690300,0.758668,0.280303,0.078570,0.280303,0.122736
1000,0.466200,0.588965,0.816288,0.819136,0.816288,0.817556
1500,0.212200,0.454569,0.880682,0.878512,0.880682,0.877078


TrainOutput(global_step=1864, training_loss=0.39220005862191, metrics={'train_runtime': 3428.3089, 'train_samples_per_second': 8.695, 'train_steps_per_second': 0.544, 'total_flos': 7842814338170880.0, 'train_loss': 0.39220005862191, 'epoch': 8.0})

In [ ]:
eval_results = trainer.evaluate()
print(f'Langugae fi: {eval_results}')

Langugae fi: {'eval_loss': 0.4704389274120331, 'eval_accuracy': 0.8882575757575758, 'eval_precision': 0.8872430723789848, 'eval_recall': 0.8882575757575758, 'eval_f1': 0.8842427349497883, 'eval_runtime': 17.0065, 'eval_samples_per_second': 31.047, 'eval_steps_per_second': 3.881, 'epoch': 8.0}


In [ ]:
# Predictions
outputs = trainer.predict(tokenized_eval_fi)
logits = outputs.predictions
labels = outputs.label_ids
predictions = np.argmax(logits, axis=-1)

print("Predictions:", predictions)
print("True Labels:", labels)

# Confusion matrix
conf_matrix = confusion_matrix(labels, predictions)

#print("Confusion Matrix:")
#print(conf_matrix)

Predictions: [1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1
 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1
 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 0
 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0
 0 0 0 0 0 1

## Japanese Language




In [ ]:
#Checking the balance

le = LabelEncoder()
labels_encoded = le.fit_transform(tokenized_train_ja['labels'])
unique, counts = np.unique(labels_encoded, return_counts=True)
class_distribution = dict(zip(le.inverse_transform(unique), counts))
print(class_distribution)

{0: 373, 1: 1928}


In [ ]:
#Oversampling minory class

df = pd.DataFrame(tokenized_train_fi)

# Counting the occurrences of each label
class_counts = df['labels'].value_counts()

majority_label = class_counts.idxmax()
minority_label = class_counts.idxmin()

n_minority_samples = class_counts[majority_label] - class_counts[minority_label]

# Oversampling the minority class
minority_samples = df[df['labels'] == minority_label]
oversampled_minority_samples = minority_samples.sample(n=n_minority_samples, replace=True, random_state=42)

# Combining datasets
balanced_df = pd.concat([df, oversampled_minority_samples])

# Shuffle to mix oversampled data
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_dataset = Dataset.from_pandas(balanced_df)

# Checking balance
new_label_counts = balanced_df['labels'].value_counts()
print(new_label_counts)


Balanced label counts:
 labels
1    1928
0    1928
Name: count, dtype: int64


In [ ]:
train_args = TrainingArguments(output_dir="my_trainer",
                                  eval_strategy="steps",
                                  num_train_epochs=3,
                                  per_device_train_batch_size=16,
                                  learning_rate=2e-5,
                                  eval_steps=500)

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset= balanced_dataset,
    eval_dataset= tokenized_eval_ja,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.292700,0.896162,0.844298,0.851192,0.844298,0.838001


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.292700,0.896162,0.844298,0.851192,0.844298,0.838001


TrainOutput(global_step=723, training_loss=0.23892407925132234, metrics={'train_runtime': 1620.12, 'train_samples_per_second': 7.14, 'train_steps_per_second': 0.446, 'total_flos': 3043668688404480.0, 'train_loss': 0.23892407925132234, 'epoch': 3.0})

In [ ]:
eval_results = trainer.evaluate()
print(f'Langugae ja: {eval_results}')

Langugae ja: {'eval_loss': 0.9059450626373291, 'eval_accuracy': 0.8464912280701754, 'eval_precision': 0.8530724545502729, 'eval_recall': 0.8464912280701754, 'eval_f1': 0.8404688906732851, 'eval_runtime': 16.0842, 'eval_samples_per_second': 28.351, 'eval_steps_per_second': 3.544, 'epoch': 3.0}


In [ ]:
# Predictions
outputs = trainer.predict(tokenized_eval_ja)
logits = outputs.predictions
labels = outputs.label_ids
predictions = np.argmax(logits, axis=-1)

print("Predictions:", predictions)
print("True Labels:", labels)

# Confusion matrix
conf_matrix = confusion_matrix(labels, predictions)

print("Confusion Matrix:")
print(conf_matrix)

Predictions: [1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0
 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1
 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0
 0 1 0 1 1 0 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1
 0 0 0 0 0 0 1 1 1 0 1 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 1 1 1 0 0 1 1 0]
True Labels: [1 1 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1